# Topic-based Text Generation in a Specific Author's Style

## Instructions to get things working

1. Clone data using the first 4 cells
2. upload a dataset to the datasets/ directory
3. if you are using a dataset that has already been processed skip to STEP ____
4. open up datasets/dataset2bpe.py and replace line 10 with 
```python
roberta = torch.hub.load('pytorch/fairseq:main', 'roberta.base')
```
5. open up style_paraphrase/utils.py and replace lines 14 and 15 with 
```python
from style_paraphrase.dataset_config import DATASET_CONFIG
from style_paraphrase.data_utils import update_config, Instance
```
6. run the code blocks under "generate paraphrases for input text"
7. run the code blocks under "train the inverse paraphraser"
8. use the model to generate stylysed text in the code blocks under "generate an output"

In [1]:
!rm -rf sample_data && \
git init && \
git remote add origin https://github.com/martiansideofthemoon/style-transfer-paraphrase.git && \
git pull origin master

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 1687, done.
remote: Counting objects: 100% (1687/1687), done.
remote: Compressing objects: 100% (1251/1251), done.
remote: Total 1687 (delta 575), reused 1463 (delta 388), pack-reused 0
Receiving objects: 100% (1687/1687), 9.69 MiB | 18.75 MiB/s, done.
Resolving deltas: 100% (575/575), done.
From https://github.com/martiansideofthemoon/style-transfer-paraphrase
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master
Wed Oct 20 23:31:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                            

In [2]:
!pip install gdown
!pip install virtualenv
!cd style_paraphrase/saved_models && \
mkdir paraphraser_gpt2_large && cd paraphraser_gpt2_large && \
gdown --id 1wC-VicaFopS6c2yYsu2fWPgskoikFnjx && \
gdown --id 1uq8yrRRy5RiT0EygS5ibzDLX8aQgzb9y && \
gdown --id 1ukB4XWBF3MyDtTLBxiZczZbirW-1Yb9x && \
gdown --id 1ptmbHKxuPMpWdpknUZcAfhFDmcQmbTGB && \
gdown --id 1BsZpKHCbLuakpeUPHZHis8xywpBpP8I6 && \
gdown --id 13AGYibZTEbnUaCGlQ6XHgqLDzvLWFgwc && \
gdown --id 1k2mJZQWP7IReA0lOVXSCTsz_kNnK2jhq && \
gdown --id 1ftLcRGkqHwcYcFz31wNYOEAfzpn19l_m && \
gdown --id 1fWJmfbUkszzJ6quLDKh92Tk6CLu3I5aI

     |████████████████████████████████| 5.3 MB 4.0 MB/s 
     |████████████████████████████████| 496 kB 42.5 MB/s 
Downloading...
From: https://drive.google.com/uc?id=1wC-VicaFopS6c2yYsu2fWPgskoikFnjx
To: /content/style_paraphrase/saved_models/paraphraser_gpt2_large/special_tokens_map.json
100% 214/214 [00:00<00:00, 376kB/s]
Downloading...
From: https://drive.google.com/uc?id=1uq8yrRRy5RiT0EygS5ibzDLX8aQgzb9y
To: /content/style_paraphrase/saved_models/paraphraser_gpt2_large/training_args.bin
100% 2.05k/2.05k [00:00<00:00, 3.21MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ukB4XWBF3MyDtTLBxiZczZbirW-1Yb9x
To: /content/style_paraphrase/saved_models/paraphraser_gpt2_large/tokenizer_config.json
100% 60.0/60.0 [00:00<00:00, 67.7kB/s]
Downloading...
From: https://drive.google.com/uc?id=1ptmbHKxuPMpWdpknUZcAfhFDmcQmbTGB
To: /content/style_paraphrase/saved_models/paraphraser_gpt2_large/vocab.json
100% 899k/899k [00:00<00:00, 56.1MB/s]
Downloading...
From: https://drive.google.com/u

In [3]:
!pwd && \
pip install torch==1.9.1 torchvision transformers && \
pip install -r requirements.txt && \
pip install --editable . && \
cd fairseq && \
pip install --editable .

/content
     |████████████████████████████████| 831.4 MB 6.9 kB/s 
     |████████████████████████████████| 2.9 MB 34.8 MB/s 
     |████████████████████████████████| 22.1 MB 1.5 MB/s 
     |████████████████████████████████| 3.3 MB 22.7 MB/s 
     |████████████████████████████████| 895 kB 76.2 MB/s 
     |████████████████████████████████| 56 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 67.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu111
    Uninstalling torchvision-0.10.0+cu111:
      Successfully uninstalled torchvision-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the 

In [4]:
!pwd

!wget https://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz
!tar -xzvf roberta.large.tar.gz

!export ROBERTA_LARGE=$PWD/roberta.large

/content
--2021-10-20 23:37:42--  https://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 655283069 (625M) [application/gzip]
Saving to: ‘roberta.large.tar.gz’

roberta.large.tar.g 100%[===================>] 624.93M  11.8MB/s    in 54s     

2021-10-20 23:38:38 (11.6 MB/s) - ‘roberta.large.tar.gz’ saved [655283069/655283069]

roberta.large/
roberta.large/dict.txt
roberta.large/model.pt
roberta.large/NOTE


## Generate paraphrases for input text

In [6]:
# first make sure you upload train,test,dev.{txt, label} to datasets/[name]
# make sure that the label files are the same length as their matching text file
# the labels should all say "original"

# change "tolstoy" to your dataset name

# *** copy below into dataset2bpe
# roberta = torch.hub.load('pytorch/fairseq:main', 'roberta.base')
# ***

!pip install regex requests hydra-core omegaconf bitarray

!python datasets/dataset2bpe.py --dataset datasets/tolstoy

# make sure dict.txt from the dataset is in the same directory as all the folders from github
!cp /content/datasets/tolstoy/dict.txt /content/
!cp /content/datasets/tolstoy/dict.txt /
!datasets/bpe2binary.sh datasets/tolstoy


Namespace(align_suffix=None, alignfile=None, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='datasets/tolstoy-bin/label', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=True, optimizer='nag', padding_factor=8, seed=1, source_lang=None, srcdict=None, target_lang=None, task='translation', tbmf_wrapper=False, tensorboard_logdir='', testpref=None, tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, trainpref='datasets/tolstoy/train.label', user_dir=None, validpref='datasets/tolstoy/dev.label', workers=24)
Traceback (most recent call last):
  File "/usr/local/bin/fairseq-preprocess", line 33, in <module>
    sys.exit(load_entry_point('fairseq', 'console_scripts', 'fairseq-pre

In [7]:
# PARAPHRASE INPUTS
# may need to go through certain files and add style_paraphrase infront of imports
# open style_paraphrase/utils.py and change lines 14 and 15 to 
# 
# from style_paraphrase.dataset_config import DATASET_CONFIG
# from style_paraphrase.data_utils import update_config, Instance

import argparse
import os
import sys
import torch
import tqdm

from style_paraphrase.inference_utils import GPT2Generator

batch_size = 64
dataset = 'datasets/tolstoy'
model_dir = "/content/style_paraphrase/saved_models/paraphraser_gpt2_large"
paraphrase_str = "paraphrase_250"
top_p_value = 0.0


roberta = torch.hub.load('pytorch/fairseq:main', 'roberta.base')
if not torch.cuda.is_available():
    print("Please check if a GPU is available or your Pytorch installation is correct.")
    sys.exit()

print("Loading paraphraser...")
paraphraser = GPT2Generator(model_dir, upper_length="same_5")
paraphraser.modify_p(top_p=top_p_value)

print("starting training")
for split in ["train", "dev", "test"]:
    data_path = os.path.join(dataset, split) + ".txt"
    label_path = os.path.join(dataset, split) + ".label"

    with open(data_path, "r") as f:
        data = f.read().strip().split("\n")

    with open(label_path, "r") as f:
        labels = f.read().strip().split("\n")

    assert len(data) == len(labels)

    outputs = []
    for i in tqdm.tqdm(range(0, len(data), batch_size), desc="minibatches of {} split done...".format(split)):
        generations, _ = paraphraser.generate_batch(data[i:i + batch_size])
        outputs.extend(generations)

    outputs = [roberta.bpe.encode(x) for x in outputs]

    output_path = os.path.join(dataset, split) + ".{}_input0.bpe".format(paraphrase_str)

    with open(output_path, "w") as f:
        f.write("\n".join(outputs) + "\n")

print("done")


Using cache found in /root/.cache/torch/hub/pytorch_fairseq_main
2021-10-20 23:44:16 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.base.tar.gz from cache at /root/.cache/torch/pytorch_fairseq/37d2bc14cf6332d61ed5abeb579948e6054e46cc724c7d23426382d11a31b2d6.ae5852b4abc6bf762e0b6b30f19e741aa05562471e9eb8f4a6ae261f04f9b350
/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:126: UserWarning: In 'config': Usage of deprecated keyword in package header '#

Loading paraphraser...


Some weights of the model checkpoint at /content/style_paraphrase/saved_models/paraphraser_gpt2_large were not used when initializing GPT2LMHeadModel: ['transformer.extra_embedding_project.weight', 'transformer.extra_embedding_project.bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
minibatches of train split done...:   0%|          | 0/478 [00:00<?, ?it/s]

starting training


minibatches of test split done...: 100%|██████████| 59/59 [04:17<00:00,  4.36s/it]


done


## Train the inverse-paraphraser

In [ ]:
# add dataset to dataset_config.py

!python -m torch.distributed.launch --nproc_per_node=1 /content/style_paraphrase/run_lm_finetuning.py \
    --output_dir=/content/style_paraphrase/saved_models/model_tolstoy_0 \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --do_train \
    --data_dir=datasets/tolstoy \
    --save_steps 1000 \
    --logging_steps 20 \
    --save_total_limit -1 \
    --evaluate_during_training \
    --num_train_epochs 3 \
    --gradient_accumulation_steps 2 \
    --per_gpu_train_batch_size 5 \
    --job_id tolstoy_0 \
    --learning_rate 5e-5 \
    --prefix_input_type paraphrase_250 \
    --global_dense_feature_list none \
    --specific_style_train 0 \
    --optimizer adam


Streaming output truncated to the last 5000 lines.
Iteration:  19% 1180/6110 [05:24<23:08,  3.55it/s]
Iteration:  19% 1181/6110 [05:24<22:08,  3.71it/s]
Iteration:  19% 1182/6110 [05:25<23:10,  3.54it/s]
Iteration:  19% 1183/6110 [05:25<22:09,  3.71it/s]
Iteration:  19% 1184/6110 [05:25<23:05,  3.55it/s]
Iteration:  19% 1185/6110 [05:25<22:07,  3.71it/s]
Iteration:  19% 1186/6110 [05:26<23:07,  3.55it/s]
Iteration:  19% 1187/6110 [05:26<22:06,  3.71it/s]
Iteration:  19% 1188/6110 [05:26<23:04,  3.55it/s]
Iteration:  19% 1189/6110 [05:26<22:05,  3.71it/s]
Iteration:  19% 1190/6110 [05:27<23:00,  3.56it/s]
Iteration:  19% 1191/6110 [05:27<22:03,  3.72it/s]
Iteration:  20% 1192/6110 [05:27<22:57,  3.57it/s]
Iteration:  20% 1193/6110 [05:28<21:59,  3.73it/s]
Iteration:  20% 1194/6110 [05:28<22:57,  3.57it/s]
Iteration:  20% 1195/6110 [05:28<21:58,  3.73it/s]
Iteration:  20% 1196/6110 [05:28<23:01,  3.56it/s]
Iteration:  20% 1197/6110 [05:29<22:02,  3.71it/s]
Iteration:  20% 1198/6110 [05:2

## Generate an output

In [ ]:
from style_paraphrase.inference_utils import GPT2Generator

top_p = 0.9
input = "I love learning about Neural networks and deep learning because they are really cool"

# paraphraser = GPT2Generator("/content/style_paraphrase/saved_models/paraphraser_gpt2_large", upper_length="eos")
# output = paraphraser.generate(input, top_p=0.7)
# print(output)

model = GPT2Generator("/content/style_paraphrase/saved_models/model_eminem_0/checkpoint-5475", upper_length="300")

transferred_output = model.generate(input, top_p=top_p)
print(transferred_output)

Just love learning about Neural Networks and deep learning 'cause they just cool stuff


## Save Data to GDRIVE

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
